# Using Python to create a work flow, using ChatGPT api to convert document to tables

### key note:
-because it is a LLM model, it doesn't work that well when the size of file is large

-ideally we should segrete tasks into smaller element

-in this particular case, pdfplumber or other similar packages might be better

## Environment setup
First, make sure the OpenAI API works...
- here i manually altered the environment used in this local file. it changes everytime I open the file

In [39]:
import sys
import os
print(sys.executable)
os.environ["OPENAI_API_KEY"] = "My key"

C:\Users\46798566\AppData\Local\anaconda3\python.exe


In [35]:
from openai import OpenAI
client = OpenAI()

In [41]:
1
print('lin')
print('lin222')

1

- simply test if the OpenAI works or not...  if this package is not installed, simply try: %pip install tiktoken

In [72]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4.1-mini")
text = "I, what am I doing" 
tokens = enc.encode(text)
print(tokens)

[40, 11, 1412, 939, 357, 5306]


In [ ]:
resp = client.responses.create(
    model="gpt-5.1",
    input="Say hello in one sentence."
)

print(resp.output_text)

 if this package is not installed, simply try: %pip install tiktoken

- choose all the files that I want to extract

In [26]:
import glob
directory = r"C:\Users\46798566/Box\Py_codes\TexasCourt\Book\Attorney_list"
den_files = (
            glob.glob(os.path.join(directory, "AttorneyPopulationDensity*.pdf")) +
            glob.glob(os.path.join(directory, "PopulationDensityReport*.pdf"))
            )
#pdf_text = open("extracted_text.txt").read()

In [7]:
# with open(upload_file, "rb") as f:
#     data = f.read()
# print(len(data))

296761


Here is the most important part, setting up the prompt! 

In [11]:
from openai import OpenAI
import pandas as pd

client = OpenAI()
prompt = """
Extract tables from the PDF.

Rules:
- Keep ONLY rows that are counties, exclude MSAs and other non related rows
- Ignore any MSA / Metro / CBSA rows.
- Extract all 254 counties, each county as a row
- Convert the result as a csv file so it is easy to convert
- Use EXACTLY these keys for each record, leave the record as NA if you can not find the record
[
  {
    "county name": "Hardin County",
    "active in state texas attorney": "59",
    "population": "1,275,648",       # here in the table in the pdf, Texas population means the population in that county
    "ratio of attorney to population": "1:508",
    "attorney as a percentage of total in state attorney",  "0.92%"
  }
]

Notes:
- “Texas population” in the table means the population in that county.

"""

# upload_file = den_files[0]

Before upload all the files, upload one and do a test:

In [13]:
# upload one file first
upload_file = den_files[6]
uploaded0 = client.files.create(
    file=open(upload_file, "rb"),
    purpose="user_data",
)

In [ ]:
resp = client.responses.create(
    model="gpt-5.2",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_file", "file_id": uploaded0.id},
            {"type": "input_text", "text": prompt},
        ],
    }]
)

In [ ]:
data = resp.output_text
df = pd.read_json(StringIO(data))

In [110]:
# see if it works...

Here I set up the OpenAI environment

In [ ]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

# upload all the legit files, as a list
uploaded_file_list = []
for upload_file in den_files[0:4]:    # only selected the first four files to save tokens....
    print('here is file', upload_file, "being uploaded")
    uploaded = client.files.create(
        file=open(upload_file, "rb"),
        purpose="user_data",
    )
    uploaded_file_list.append(uploaded)
    


In [ ]:
#upload_file = uploaded_file_list[0]
response_list = []
table_list = []
for uploaded_file in uploaded_file_list:
    resp = client.responses.create(
        model="gpt-5.2",
        input=[{
            "role": "user",
            "content": [
                {"type": "input_file", "file_id": uploaded_file.id},
                {"type": "input_text", "text": prompt},
            ],
        }],
    )
    response_list.append(resp)
    print('We have got our response for file',uploaded_file.filename,'back!')

    # the following lines, hopefully will extract the data within that table and convert them to a data frame
    data = resp.output_text
    df = pd.read_json(StringIO(data))
    

In [ ]:
data = json.loads( response_list[3].output_text )
print(data)
pd.DataFrame(data)
df = pd.DataFrame(data["data_found_in_provided_content"])


In [75]:
response_list[0].output_text

'[\n  {\n    "county name": null,\n    "active in state texas attorney": null,\n    "population": null,\n    "ratio of attorney to population": null,\n    "attorney as a percentage of total in state attorney": null\n  }\n]'

In [ ]:
response_list[0] = data_found_in_provided_content

In [19]:
# Wrap the literal string in a StringIO object
# This makes pandas read the string as if it were a file
import pandas as pd
from io import StringIO
import json

try:
    df = pd.read_json(StringIO(data))
except ValueError as e:
    # If the JSON is not valid (e.g. single quotes), you might need json.loads first
    # and then create a DataFrame
    data = json.loads(json_string_data)
    df = pd.DataFrame(data)

df.to_csv("county_stats.csv", index=False)

In [21]:
sys.path

['C:\\Users\\46798566\\AppData\\Local\\anaconda3\\python313.zip',
 'C:\\Users\\46798566\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\46798566\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\46798566\\AppData\\Local\\anaconda3',
 '',
 'C:\\Users\\46798566\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\46798566\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\46798566\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\46798566\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin']

## Advantage: 

# what I am doing here

## Disdvantage: 
- not sure why it is always slower than manual gpt input